In [ ]:
########## Step1: 데이터 전처리
##### 1-1. 한글 처리 konlp와 Mecab 설치
# https://velog.io/@heiswicked/M1-Part11-%EB%B3%B5%EB%B6%88%EB%B3%B5%EC%84%A4%EC%B9%98-konlpy.tag-MECAB-on-M1-ver.221230
!pip install konlpy #  한국어 정보처리 파이썬 패키지 "코 엔 엘 파이"
!pip install mecab-python # 형태소 분석기 Mecab
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 9.0 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=5c09fed46ee236be021f00c7f8c57f087f2fe2fac9a40977817bbeb2f9e5f422
  Stored in directory: /root/.cache/pip/wheels/62/0e/84/39e1c8bf2f2da63964fe43fb2932027d6344f8766edf28bc8c
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  2672k      0 --:--:-- --:--:-- --:--:-- 2672k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/exampl

In [ ]:
#1-1-(1) Mecab 사용 예제
from konlpy.tag import Mecab
mecab=Mecab()
mecab.morphs("가장 맛있는 맛집은 어디인가요?")

['가장', '맛있', '는', '맛집', '은', '어디', '인가요', '?']

In [ ]:
##### 1-2. 전처리에 필요한 패키지, 클래스, 함수 불러오기
import pandas as pd; import numpy as np
import urllib.request
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
##### 1-3. 데이터(네이버 영화 리뷰) 불러오기
url="https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
urllib.request.urlretrieve(url, filename="train_pd.txt")
train_pd = pd.read_table('train_pd.txt')#150,000개

In [ ]:
train_pd

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
##### 1-4. (1)NaN, (2)한글 아닌 것, (3)빈 문자열, (4)중복 제거
import warnings
warnings.filterwarnings('ignore')

#(1) NaN 제거
train_pd = train_pd.dropna()

#(2) non한글 제거
pat, repl='[^ㄱ-ㅎㅏ-ㅣ가-힣]', ''
train_pd['document'] = train_pd['document'].str.replace(pat,repl)

#(3) 빈 문자열  제거
train_pd=train_pd[train_pd['document'].str.strip().astype(bool)]

#(4) 중복 제거
train_pd.drop_duplicates(subset='document',inplace=True)

In [ ]:
train_pd

,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소이야기구먼솔직히재미는없다평점조정,0
4,6483659,사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...,1
...,...,...,...
149994,9458520,디케이드다음에더블다음에오즈인데더블은조금밖에안나오네요,1
149995,6222902,인간이문제지소는뭔죄인가,0
149997,9311800,이게뭐요한국인은거들먹거리고필리핀혼혈은착하다,0
149998,2376369,청춘영화의최고봉방황과우울했던날들의자화상,1


In [ ]:
##### 1-5. 리뷰를 문장2단어나열로 변환(단어 토큰화)
#불용어 집합 stopwards
stopwords =['가', '걍', '과', '는', '도', '들', '를', '에', '와','으로',
            '은','을', '의', '이', '자', '잘', '좀', '하다', '한',  '한다']

mecab = Mecab()
trainX_morphs = []

for doc in train_pd['document']:
  morphs = mecab.morphs(doc)
  morphs = [word for word in morphs if not word in stopwords]
  trainX_morphs.append(morphs)

print(train_pd["document"][0])
print(trainX_morphs[0])
print('리뷰 개수 = ',len(trainX_morphs))

아 더빙.. 진짜 짜증나네요 목소리
['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']
리뷰 개수 =  146182


In [ ]:
trainX_morphs

[['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠',
  '.',
  '..',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '.',
  '...',
  '오버',
  '연기',
  '조차',
  '가볍',
  '지',
  '않',
  '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다'],
 ['교도소', '이야기', '구먼', '.', '.', '솔직히', '재미', '없', '다', '.', '.', '평점', '조정'],
 ['사이몬페그',
  '익살',
  '스런',
  '연기',
  '돋보였',
  '던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '했',
  '던',
  '커스틴',
  '던스트',
  '너무나',
  '이뻐',
  '보였',
  '다'],
 ['막',
  '걸음마',
  '뗀',
  '3',
  '세',
  '부터',
  '초등',
  '학교',
  '1',
  '학년',
  '생',
  '인',
  '8',
  '살',
  '용',
  '영화',
  '.',
  'ㅋㅋㅋ',
  '.',
  '..',
  '별반',
  '개',
  '아까움',
  '.'],
 ['원작', '긴장감', '제대로', '살려', '내', '지', '못했', '다', '.'],
 ['별',
  '반개',
  '아깝',
  '다',
  '욕',
  '나온다',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '몇',
  '년',
  '인지',
  '.',
  '.',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫',
  '겟',
  '다',
  '납치',
  '.',
  '감금',
  '만',
  '반복',
  '반복',
  '.',
  '.',
  '드라마',
  '가족',
  '없',
  '다',
  

In [ ]:
test_trainX_morphs = trainX_morphs[:10]
print(test_trainX_morphs)
print(len(test_trainX_morphs))

[['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], ['흠', '.', '..', '포스터', '보고', '초딩', '영화', '줄', '.', '...', '오버', '연기', '조차', '가볍', '지', '않', '구나'], ['너무', '재', '밓었다그래서보는것을추천한다'], ['교도소', '이야기', '구먼', '.', '.', '솔직히', '재미', '없', '다', '.', '.', '평점', '조정'], ['사이몬페그', '익살', '스런', '연기', '돋보였', '던', '영화', '!', '스파이더맨', '에서', '늙', '어', '보이', '기', '만', '했', '던', '커스틴', '던스트', '너무나', '이뻐', '보였', '다'], ['막', '걸음마', '뗀', '3', '세', '부터', '초등', '학교', '1', '학년', '생', '인', '8', '살', '용', '영화', '.', 'ㅋㅋㅋ', '.', '..', '별반', '개', '아까움', '.'], ['원작', '긴장감', '제대로', '살려', '내', '지', '못했', '다', '.'], ['별', '반개', '아깝', '다', '욕', '나온다', '이응경', '길용우', '연기', '생활', '몇', '년', '인지', '.', '.', '정말', '발', '로', '해도', '그것', '보단', '낫', '겟', '다', '납치', '.', '감금', '만', '반복', '반복', '.', '.', '드라마', '가족', '없', '다', '연기', '못', '하', '사람', '만', '모엿', '네'], ['액션', '없', '는데', '재미', '있', '몇', '안', '되', '영화'], ['왜', '케', '평점', '낮', '건데', '?', '꽤', '볼', '만한데', '.', '.', '헐리우드', '식', '화려', '함', '만', '너무', '길들여져', '있', '나'

In [ ]:
##### 1-6-1(모든 단어 사용) word_counts 만들기
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(test_trainX_morphs)
print('[모든 단어 사용]')
print('(단어, 빈도수) =',tokenizer1.word_counts)
print('단어장 =',tokenizer1.word_index)
print('단어장 =',len(tokenizer1.word_index))

[모든 단어 사용]
(단어, 빈도수) = OrderedDict({'아': 1, '더': 1, '빙': 1, '.': 19, '진짜': 1, '짜증': 1, '나': 2, '네요': 1, '목소리': 1, '흠': 1, '..': 2, '포스터': 1, '보고': 1, '초딩': 1, '영화': 4, '줄': 1, '...': 1, '오버': 1, '연기': 4, '조차': 1, '가볍': 1, '지': 2, '않': 1, '구나': 1, '너무': 2, '재': 1, '밓었다그래서보는것을추천한다': 1, '교도소': 1, '이야기': 1, '구먼': 1, '솔직히': 1, '재미': 2, '없': 3, '다': 6, '평점': 2, '조정': 1, '사이몬페그': 1, '익살': 1, '스런': 1, '돋보였': 1, '던': 2, '!': 1, '스파이더맨': 1, '에서': 1, '늙': 1, '어': 1, '보이': 1, '기': 1, '만': 4, '했': 1, '커스틴': 1, '던스트': 1, '너무나': 1, '이뻐': 1, '보였': 1, '막': 1, '걸음마': 1, '뗀': 1, '3': 1, '세': 1, '부터': 1, '초등': 1, '학교': 1, '1': 1, '학년': 1, '생': 1, '인': 1, '8': 1, '살': 1, '용': 1, 'ㅋㅋㅋ': 1, '별반': 1, '개': 1, '아까움': 1, '원작': 1, '긴장감': 1, '제대로': 1, '살려': 1, '내': 1, '못했': 1, '별': 1, '반개': 1, '아깝': 1, '욕': 1, '나온다': 1, '이응경': 1, '길용우': 1, '생활': 1, '몇': 2, '년': 1, '인지': 1, '정말': 1, '발': 1, '로': 1, '해도': 1, '그것': 1, '보단': 1, '낫': 1, '겟': 1, '납치': 1, '감금': 1, '반복': 2, '드라마': 1, '가족': 1, '못': 1, '하': 1, '사람': 1, '모엿'

In [ ]:
tokenizer1.texts_to_sequences(test_trainX_morphs)

[[18, 19, 20, 1, 1, 21, 22, 7, 23, 24],
 [25, 1, 8, 26, 27, 28, 3, 29, 1, 30, 31, 4, 32, 33, 9, 34, 35],
 [10, 36, 37],
 [38, 39, 40, 1, 1, 41, 11, 6, 2, 1, 1, 12, 42],
 [43,
  44,
  45,
  4,
  46,
  13,
  3,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  5,
  54,
  13,
  55,
  56,
  57,
  58,
  59,
  2],
 [60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  3,
  1,
  75,
  1,
  8,
  76,
  77,
  78,
  1],
 [79, 80, 81, 82, 83, 9, 84, 2, 1],
 [85,
  86,
  87,
  2,
  88,
  89,
  90,
  91,
  4,
  92,
  14,
  93,
  94,
  1,
  1,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  2,
  103,
  1,
  104,
  5,
  15,
  15,
  1,
  1,
  105,
  106,
  6,
  2,
  4,
  107,
  108,
  109,
  5,
  110,
  111],
 [112, 6, 113, 11, 16, 14, 114, 115, 3],
 [116,
  117,
  12,
  118,
  119,
  17,
  120,
  121,
  122,
  1,
  1,
  123,
  124,
  125,
  126,
  5,
  10,
  127,
  16,
  7,
  17]]

In [ ]:
### 1-6-2(tokenizer.word_counts에서 count 3 이상인 단어만 사용)
# word_counts, word_index(단어장) 만들기

# 단어_빈도수 모임에서 빈도수 2 이상인 단어만 추출
words_with_min_freq = \
        {word for word, count in tokenizer1.word_counts.items() if count >= 2}

# trainX_morphs에서 words_with_min_freq에 있는 단어만 추출
filtered_trainX_morphs = \
         [ [word for word in morphs if word in words_with_min_freq]
           for morphs in test_trainX_morphs]

# filtered_trainX_morphs으로 word_counts와 word_index(단어장) 만들기
tokenizer2 = Tokenizer(num_words=len(words_with_min_freq))
tokenizer2.fit_on_texts(filtered_trainX_morphs)
print('[빈도수 2 이상인 단어만 사용]')
print('(단어, 빈도수) =',tokenizer2.word_counts)
print('단어장 =',tokenizer2.word_index)
print('단어장 크기 =',len(tokenizer2.word_index))

[빈도수 2 이상인 단어만 사용]
(단어, 빈도수) = OrderedDict({'.': 19, '나': 2, '..': 2, '영화': 4, '연기': 4, '지': 2, '너무': 2, '재미': 2, '없': 3, '다': 6, '평점': 2, '던': 2, '만': 4, '몇': 2, '반복': 2, '있': 2, '?': 2})
단어장 = {'.': 1, '다': 2, '영화': 3, '연기': 4, '만': 5, '없': 6, '나': 7, '..': 8, '지': 9, '너무': 10, '재미': 11, '평점': 12, '던': 13, '몇': 14, '반복': 15, '있': 16, '?': 17}
단어장 크기 = 17


In [ ]:
tokenizer2.texts_to_sequences(filtered_trainX_morphs)

[[1, 1, 7],
 [1, 8, 3, 1, 4, 9],
 [10],
 [1, 1, 11, 6, 2, 1, 1, 12],
 [4, 13, 3, 5, 13, 2],
 [3, 1, 1, 8, 1],
 [9, 2, 1],
 [2, 4, 14, 1, 1, 2, 1, 5, 15, 15, 1, 1, 6, 2, 4, 5],
 [6, 11, 16, 14, 3],
 [12, 1, 1, 5, 10, 16, 7]]

In [ ]:
##### 1-6-1(모든 단어 사용) word_counts 만들기
tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainX_morphs)
print('[모든 단어 사용]')
print('(단어, 빈도수) =',tokenizer.word_counts)

[모든 단어 사용]
(단어, 빈도수) = OrderedDict({'아': 9115, '더': 5144, '빙': 272, '.': 159415, '진짜': 8301, '짜증': 1477, '나': 12751, '네요': 9003, '목소리': 370, '흠': 242, '..': 22655, '포스터': 568, '보고': 836, '초딩': 421, '영화': 57345, '줄': 2902, '...': 5306, '오버': 133, '연기': 6833, '조차': 315, '가볍': 249, '지': 19237, '않': 7725, '구나': 796, '너무': 10974, '재': 1669, '밓었다그래서보는것을추천한다': 1, '교도소': 12, '이야기': 2191, '구먼': 40, '솔직히': 1079, '재미': 4066, '없': 15804, '다': 54833, '평점': 6247, '조정': 45, '사이몬페그': 2, '익살': 11, '스런': 221, '돋보였': 60, '던': 5903, '!': 19269, '스파이더맨': 62, '에서': 8070, '늙': 201, '어': 8802, '보이': 888, '기': 8086, '만': 11823, '했': 8134, '커스틴': 4, '던스트': 2, '너무나': 708, '이뻐': 52, '보였': 121, '막': 430, '걸음마': 2, '뗀': 3, '3': 2605, '세': 730, '부터': 1963, '초등': 143, '학교': 397, '1': 5442, '학년': 130, '생': 424, '인': 6842, '8': 841, '살': 1835, '용': 1210, 'ㅋㅋㅋ': 5311, '별반': 85, '개': 2801, '아까움': 370, '원작': 1329, '긴장감': 828, '제대로': 820, '살려': 100, '내': 6318, '못했': 325, '별': 1541, '반개': 157, '아깝': 2384, '욕': 715, '나온다': 3

In [ ]:
### 1-6-2(tokenizer.word_counts에서 count 3 이상인 단어만 사용)
# word_counts, word_index(단어장) 만들기

# 단어_빈도수 모임에서 빈도수 3 이상인 단어만 추출
words_with_min_freq = \
        {word for word, count in tokenizer.word_counts.items() if count >= 3}

# trainX_morphs에서 words_with_min_freq에 있는 단어만 추출
filtered_trainX_morphs = \
         [ [word for word in morphs if word in words_with_min_freq]
           for morphs in trainX_morphs]

# filtered_trainX_morphs으로 word_counts와 word_index(단어장) 만들기
tokenizer = Tokenizer(num_words=len(words_with_min_freq))
tokenizer.fit_on_texts(filtered_trainX_morphs)
print('[빈도수 3 이상인 단어만 사용]')
print('(단어, 빈도수) =',tokenizer.word_counts)
print('단어장 =',tokenizer.word_index)
print('단어장 크기 =',len(tokenizer.word_index))

[빈도수 3 이상인 단어만 사용]
(단어, 빈도수) = OrderedDict({'아': 9115, '더': 5144, '빙': 272, '.': 159415, '진짜': 8301, '짜증': 1477, '나': 12751, '네요': 9003, '목소리': 370, '흠': 242, '..': 22655, '포스터': 568, '보고': 836, '초딩': 421, '영화': 57345, '줄': 2902, '...': 5306, '오버': 133, '연기': 6833, '조차': 315, '가볍': 249, '지': 19237, '않': 7725, '구나': 796, '너무': 10974, '재': 1669, '교도소': 12, '이야기': 2191, '구먼': 40, '솔직히': 1079, '재미': 4066, '없': 15804, '다': 54833, '평점': 6247, '조정': 45, '익살': 11, '스런': 221, '돋보였': 60, '던': 5903, '!': 19269, '스파이더맨': 62, '에서': 8070, '늙': 201, '어': 8802, '보이': 888, '기': 8086, '만': 11823, '했': 8134, '커스틴': 4, '너무나': 708, '이뻐': 52, '보였': 121, '막': 430, '뗀': 3, '3': 2605, '세': 730, '부터': 1963, '초등': 143, '학교': 397, '1': 5442, '학년': 130, '생': 424, '인': 6842, '8': 841, '살': 1835, '용': 1210, 'ㅋㅋㅋ': 5311, '별반': 85, '개': 2801, '아까움': 370, '원작': 1329, '긴장감': 828, '제대로': 820, '살려': 100, '내': 6318, '못했': 325, '별': 1541, '반개': 157, '아깝': 2384, '욕': 715, '나온다': 366, '이응경': 8, '길용우': 4, '생활': 146, '몇': 1114,

#**12주차 2번째(1시간) 여기까지**

#**<font color=blue>13주차 1번째(2시간) 시작</font>**

In [ ]:
### 1-6-3. 문장2단어나열을 문장2단어수열로 변환하여 저장(train_int)
print("문장2단어나열:",filtered_trainX_morphs[0])

trainX_int = tokenizer.texts_to_sequences(filtered_trainX_morphs)

print("문장2단어수열:", trainX_int[0])

문장2단어나열: ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']
문장2단어수열: [28, 70, 929, 1, 1, 35, 224, 16, 29, 738]


In [ ]:
# 불용어와 빈도수 3 이상 처리 과정에서 빈 문장2단어나열 존재
# 따라서 빈 문장2단어수열 존재
print(trainX_int[10])

[1, 35, 225, 3, 529]


In [ ]:
### 1-6-4. Non-empty 문장2단어수열과 label 저장(trainX_int, trainY_np)
# 빈 문장2단어수열 인덱스
index_emptyDoc=[ i for i,doc in enumerate(trainX_int) if doc ==[] ]

#(trainX_int, trainY_np)에서 빈 문장2단어수열 제거
trainX_int = np.array(trainX_int, dtype=object)  # dtype=object로 리스트를 포함하는 배열 생성
trainX_int = np.delete(trainX_int, index_emptyDoc, axis=0)

trainY_np = np.array(train_pd['label'])
trainY_np = np.delete(trainY_np, index_emptyDoc, axis=0)

print(trainX_int[10])
print(len(trainX_int))

[1, 35, 225, 3, 529]
145917


In [ ]:
### 1-6-5. 서로 다른 문장2단어수열과 label 저장(trainX_int, trainY_np)
# 서로 다른 문장2단어수열 인덱스
_,idx=np.unique(trainX_int, return_index=True)

#(trainX_int, trainY_np)에서 서로 다른 문장2단어수열 추출
trainX_int=trainX_int[idx]
trainY_np=trainY_np[idx]

print(len(trainX_int))
print(trainY_np.shape)

145217
(145217,)


In [ ]:
##### 1-7. 0 라벨과 1 라벨 개수가 비슷한지 확인
unique, counts=np.unique(trainY_np, return_counts=True)
print(np.asarray((unique, counts)))

[[    0     1]
 [72876 72341]]


In [ ]:
##### 1-8. 각 문장2단어수열의 숫자 개수가 같도록 만들기(패딩)
print('패딩 전=',trainX_int[1])
doc_length=np.array([len(doc) for doc in trainX_int])
pad_length=np.percentile(doc_length,90)
trainX_pad = pad_sequences(trainX_int, maxlen = np.int32(pad_length))
print('패딩 후=',trainX_pad[1])

############# END: 데이터 전처리(Data Preparation) ###################

패딩 전= [1, 1]
패딩 후= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]


In [ ]:
pad_length
len(trainX_pad[1])
doc_length
np.max(doc_length)

np.int64(116)

In [ ]:
########## Step2: 학습 데이터 train_dataloader

In [ ]:
### 2-1. numpy 자료형을 tensor 자료형으로 변환
import torch
trainX = torch.from_numpy(trainX_pad)
trainY = torch.from_numpy(trainY_np)

In [ ]:
### 2-2. batch_size=2로 학습 데이터 분할
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(trainX, trainY)
train_dataloader = DataLoader(train_dataset, batch_size=60, shuffle=False)

In [ ]:
########## Step3: 신경망 정의
import torch; import torch.nn as nn; torch.manual_seed(1)

class Model(nn.Module):

  def __init__(self, vocab_size, embed_dim, hidden_size):
    super().__init__()
    self.embed=nn.Embedding(vocab_size, embed_dim)
    self.rnn=nn.RNN(embed_dim, hidden_size, batch_first=True)
    self.fc=nn.Linear(hidden_size, 1)

  def forward(self, input):
    embed_out = self.embed(input)
    rnn_down, rnn_right=self.rnn(embed_out)
    output = self.fc(rnn_down[:, :, -1])
    return output.squeeze()

model=Model(vocab_size=len(tokenizer.word_index), embed_dim=100,
            hidden_size=int(pad_length))

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
len(train_dataloader)

2421

In [58]:
########## Step3: 신경망 정의
import torch; import torch.nn as nn; torch.manual_seed(1)

class Model(nn.Module):

  def __init__(self, vocab_size, embed_dim, hidden_size):
    super().__init__()
    self.embed=nn.Embedding(vocab_size + 1, embed_dim) # 패딩 인덱스(0)를 수용하기 위해 +1 추가
    self.rnn=nn.RNN(embed_dim, hidden_size, batch_first=True)
    self.fc=nn.Linear(hidden_size, 1)

  def forward(self, input):
    embed_out = self.embed(input)
    rnn_down, rnn_right=self.rnn(embed_out)
    output = self.fc(rnn_down[:, :, -1])
    return output.squeeze()

# 필요한 경우 패딩 및 OOV를 포함하도록 vocab_size 조정
vocab_size = len(tokenizer.word_index) + 1  # 패딩을 위해 +1 추가

model=Model(vocab_size=vocab_size, embed_dim=100,
            hidden_size=int(pad_length))

In [61]:
class Model(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.RNN(embed_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input):
        embed_out = self.embed(input)
        rnn_out, _ = self.rnn(embed_out)
        output = self.fc(rnn_out[:, -1, :])  # 🔥 핵심 수정
        return output.squeeze(1)


In [62]:
########## [DEBUG] Embedding index 범위 확인 ##########
for batch_input, batch_target in train_dataloader:
    print("input max:", batch_input.max().item())
    print("input min:", batch_input.min().item())
    print("embedding size:", model.embed.num_embeddings)
    break


input max: 21680
input min: 0
embedding size: 22568


In [53]:
vocab_size = tokenizer.num_words


In [63]:
##### Step4. 학습
optimizer=torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

device = next(model.parameters()).device

for epoch in range(1):
  for input, target in train_dataloader:
        optimizer.zero_grad()

  # 장치와 dtype 변환
        input = input.to(device)
        target = target.to(device).to(model(input).dtype)  # dtype 맞춤

        output = model(input)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

print("손실=",loss.item())

손실= 0.45408400893211365


In [64]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

device = next(model.parameters()).device

for epoch in range(1):
    for input, target in train_dataloader:
        optimizer.zero_grad()

        input = input.to(device)
        target = target.to(device).float()   # 🔥 핵심 수정

        output = model(input)
        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

print("손실 =", loss.item())


손실 = 0.4687127470970154
